In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Online shoe retailer **FLO** wants to divide up its customers into different groups and develop marketing tactics for each group. To do this, the customer behaviors will be identified,
and groups will be created based on clusters in these behaviors.

In the first step, **RFM analysis** will be performed.

# RFM ANALYSIS

*  Data Understanding
*  Data Preparation
*  Calculating RFM Metrics
*  Calculating RFM Scores
*  Creating & Analysing RFM Segments

**Variables**
*  master_id: Unique customer number
*  order_channel: Which channel of the shopping platform is used (Android, ios, Desktop, Mobile)
*  last_order_channel: The channel where the most recent purchase was made
*  first_order_date: Date of the customer's first purchase
*  last_order_date: Date of the customer's last purchase
*  last_order_date_online: Date of the customer's last purchase on the online platform
*  last_order_date_offline: Date of the customer's last purchase on the offline platform
*  order_num_total_ever_online: Number of purchases made by the customer on the online platform
*  order_num_total_ever_offline: Number of purchases made by the customer on the offline platform
*  customer_value_total_ever_offline: The total fee paid by the customer for their offline shopping
*  customer_value_total_ever_online: The total fee paid by the customer for their online shopping
*  interested_in_categories_12: List of categories the customer has shopped in the last 12 months

## Data Understanding

In [ ]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_ = pd.read_csv("../input/flo-data-set/flo_data_20k.csv")
df = df_.copy()

In [ ]:
# Data overview
def check_df(dataframe, head=10):
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Variables #####################")
    print(dataframe.columns)
    print("##################### Descriptive Stats #####################")
    print(dataframe.describe().T)
    print("##################### Null Values #####################")
    print(dataframe.isnull().sum())
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Info #####################")
    print(dataframe.info())
    
check_df(df)

## Data Preparation

In [ ]:
#Omnichannel means that customers shop from both online and offline platforms.
df["Omnichannel_order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["Omnichannel_customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
#Analysis of variable types
df.info()

In [ ]:
# Convert the type of variables that express date to date type.
df.loc[:, df.columns.str.contains("date")] = df.loc[:, df.columns.str.contains("date")].apply(pd.to_datetime)
df.info()

In [ ]:
# Variable distribution and visualizations
## Categorical variable
def cat_summary(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show(block=True)
#According to the result of the relevant function, we can visually and
#numerically see from which channels customers shop more.     
cat_summary(df, "order_channel", plot=True)

In [ ]:
## Numeric variable
def num_summary(dataframe, numerical_col, plot=False):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numerical_col].describe(quantiles).T)
    print("#"*9)
    if plot:
        dataframe[numerical_col].hist()
        plt.xlabel(numerical_col)
        plt.title(numerical_col.upper())
        plt.show(block=True)
# According to the result of the relevant function
# , we can see the total number of purchases and the fee paid by the customers as a result of their purchases.
num_summary(df,"Omnichannel_order_num_total", plot=True)
num_summary(df,"Omnichannel_customer_value_total", plot=True)

Average and sum of total number of orders and paid price for each order channel

In [ ]:
df.groupby('order_channel').agg({"master_id": "count",
                                "Omnichannel_order_num_total": ["mean", "sum"],
                                 "Omnichannel_customer_value_total": ["mean", "sum"]})

In [ ]:
#Sorted values
df.sort_values("Omnichannel_customer_value_total", ascending=False).head(10)

In [ ]:
df.sort_values("Omnichannel_order_num_total", ascending=False).head(10)

In [ ]:
# we turn everything we do into function
def data_prep(dataframe, cat_variables=[], num_variables =[], sort_values =[] , date_str = "date", head = 10, plot=False):
    
    dataframe["Omnichannel_order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["Omnichannel_customer_value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    
    check_df(dataframe, head)
    dataframe.loc[:, dataframe.columns.str.contains(date_str)] = dataframe.loc[:, dataframe.columns.str.contains(date_str)].apply(pd.to_datetime)

    for col in cat_variables:
        cat_summary(dataframe, col, plot)

    for col in num_variables:
        num_summary(dataframe, col, plot)

    for col in sort_values:
        print("*"*9)
        print(f"#### sorted by {col} ####")
        print(dataframe.sort_values(col, ascending=False).head(head))

    return dataframe

# Calculating RFM Metrics

In [ ]:
# We set the analysis time to two day after the last order date
df["last_order_date"].max()
today_date = dt.datetime(2021, 6, 1)

# We do grouping for each customer to get the RFM values
rfm = df.groupby('master_id').agg({'last_order_date': lambda last_order_date: (today_date - last_order_date).dt.days,#ilgili müşterinin son satın alma tarihini bugünden çıkar gün cinsinden ver
                                     'Omnichannel_order_num_total': lambda Omnichannel_order_num_total: Omnichannel_order_num_total,
                                     'Omnichannel_customer_value_total': lambda Omnichannel_customer_value_total: Omnichannel_customer_value_total})
rfm.head()
rfm.columns = ['recency', 'frequency', 'monetary']

rfm.describe().T

# Calculating RFM Scores

In [ ]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

# Creating & Analysing RFM Segments

In [ ]:
# Customers are segmented based on standardized values.
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

rfm_final = rfm.merge(df, on="master_id", how="left")

In [ ]:
# Mean and count values of RFM values of segmented customers
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

Scenario 1:
FLO incorporates a new brand of women's shoes into its structure. The product prices of the brand it includes are above the general customer preferences. Therefore, it is desirable to contact the customers in the profile that will be interested in the promotion of the brand and the sales of the product. Customers to be contacted privately from loyal customers (champions, loyal_customers) and people who shop from the female category.

In [ ]:
# target_segments_customer_ids = rfm[rfm["segment"].isin(["champions", "loyal_customers"])]["customer_id"]
# cust_ids = \
# df[(df["master_id"].isin(target_segments_customer_ids)) & (df["interested_in_categories_12"].str.contains("KADIN"))]["master_id"]

In [ ]:
new_df = pd.DataFrame()
new_df["loyals_id"] = rfm_final[(rfm_final["interested_in_categories_12"].str.contains("KADIN")) &
          ((rfm_final["segment"] == "champions") | (rfm_final["segment"] == "loyal_customers"))]["master_id"]

In [ ]:
new_df.head(5)

Scenario 2:
Up to 40% discount is planned for Men's and Children's products.Customers who are loyal in the past and have not shopped for a long time will be the ones we are going to focus on.Those who are asleep, and new customers are specifically targeted.

In [ ]:
new_df_2 = pd.DataFrame()
new_df_2["customers_discounted"] = rfm_final[rfm_final["interested_in_categories_12"].str.contains('ERKEK|COCUK') & ((rfm_final["segment"] == "hibernating") | (rfm_final["segment"] == "cant_loose") |
                                                (rfm_final["segment"] == "new_customers"))]["master_id"]

In [ ]:
new_df_2.head(5)

In [ ]:
default_time = df["last_order_date"].max() + dt.timedelta(days=2)
def RFM_analysis(dataframe, analysis_time = default_time):
    rfm = dataframe.groupby('master_id').agg({'last_order_date': lambda last_order_date: (analysis_time - last_order_date).dt.days,
                                     'Omnichannel_order_num_total': lambda Omnichannel_order_num_total: Omnichannel_order_num_total,
                                     'Omnichannel_customer_value_total': lambda Omnichannel_customer_value_total: Omnichannel_customer_value_total})
    rfm.head()
    rfm.columns = ['recency', 'frequency', 'monetary']
    rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
    rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }
    rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)
    rfm_final = rfm.merge(dataframe, on="master_id", how="left")
    return rfm_final[["recency", "frequency", "monetary", "recency_score",
                     "frequency_score", "monetary_score", "RF_SCORE",
                     "segment"]]

# CLTV Prediction

*  Data Understanding
*  Data Preparation
*  BG-NBD Model
*  Gamma-Gamma Model
*  CLTV Prediction
*  Segmentation of Customers by CLTV

FLO wants to set a roadmap for its sales and marketing activities. In order for the company to make a medium-long-term plan,it is necessary to estimate the potential value that the existing customers will provide to the future company.

In [ ]:
!pip install lifetimes

In [ ]:
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [ ]:
df_ = pd.read_csv("../input/flo-data-set/flo_data_20k.csv")
df = df_.copy()

In [ ]:
check_df(df)

In [ ]:
df.describe([0,0.25,0.5,0.75,0.8,0.9,0.95,0.98,0.99]).T

Standardization was done when calculating the RFM score in the RFM analysis. Since CLTV estimation is a different study and there is no such standardization, we may need to perform an outlier analysis and remove the outliers from the data set.

Since the 25% quarterly and minimum values of the number of online and offline orders are the same, suppressing the lower limits for these variables will not change much. It is seen that the outlier values are much higher at the upper limits. We choose %99 as upper limit as it can be seen that the main difference starts from that point

## Data Preparation

In [ ]:
# Outlier Detection
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    up_limit = round(up_limit)
    low_limit = round(low_limit)
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable, low_limit=False):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    if low_limit:
        dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [ ]:
for col in df[["order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline",
               "customer_value_total_ever_online"]]:
    replace_with_thresholds(df, col)

In [ ]:
#Omnichannel means that customers shop from both online and offline platforms.
df["Omnichannel_order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["Omnichannel_customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [ ]:
# Convert the type of variables that express date to date type.
df.loc[:, df.columns.str.contains("date")] = df.loc[:, df.columns.str.contains("date")].apply(pd.to_datetime)

In [ ]:
# Date of analysis
df["last_order_date"].max()
today_date = dt.datetime(2021, 6, 1)

# Preparing CLTV dataframe
cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] = (df["last_order_date"] - df["first_order_date"]).dt.days
cltv_df['T_weekly'] = (today_date - df["first_order_date"]).dt.days
cltv_df['frequency'] = df["Omnichannel_order_num_total"]
cltv_df["monetary_cltv_avg"] = df["Omnichannel_customer_value_total"] / cltv_df["frequency"]

cltv_df = cltv_df[(cltv_df['frequency'] > 1)]
cltv_df["recency_cltv_weekly"] = cltv_df["recency_cltv_weekly"] / 7
cltv_df["T_weekly"] = cltv_df["T_weekly"] / 7

## BG-NBD Model

In [ ]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

#Estimate expected purchases from customers within 3 months
cltv_df["exp_sales_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4 * 3,
                                                        cltv_df['frequency'],
                                                        cltv_df['recency_cltv_weekly'],
                                                        cltv_df['T_weekly'])

#Estimate expected purchases from customers within 6 months
cltv_df["exp_sales_6_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4 * 6,
                                                        cltv_df['frequency'],
                                                        cltv_df['recency_cltv_weekly'],
                                                        cltv_df['T_weekly'])

In [ ]:
# Visual
plot_period_transactions(bgf)

## Gamma-Gamma Model

In [ ]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])

cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary_cltv_avg'])

cltv_df.index = cltv_df["customer_id"]
cltv_df.drop("customer_id", inplace=True, axis=1)

## CLTV Prediction

In [ ]:
cltv_df["cltv"] = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time=6, # 6 month
                                   freq="W",
                                   discount_rate=0.01)

cltv_df.sort_values(by="cltv", ascending=False).head(20)

## Segmentation of Customers by CLTV

In [ ]:
cltv_df["segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])

In [ ]:
# Could there be less segments
cltv_df.groupby("segment").agg({"exp_average_value": "mean",
                                "cltv": "mean",
                               "exp_sales_6_month" : "mean",
                               "frequency": "mean",
                               "monetary_cltv_avg": "mean",
                               "T_weekly": "mean",
                                "recency_cltv_weekly": "mean"})

In [ ]:
default_time = df["last_order_date"].max() + dt.timedelta(days=2)
def CLTV_Prediction(dataframe, analysis_time = default_time, projection_time = 6):
    cltv_df = pd.DataFrame()
    cltv_df["customer_id"] = dataframe["master_id"]
    cltv_df["recency_cltv_weekly"] = (dataframe["last_order_date"] - dataframe["first_order_date"]).dt.days
    cltv_df['T_weekly'] = (today_date - dataframe["first_order_date"]).dt.days
    cltv_df['frequency'] = dataframe["Omnichannel_order_num_total"]
    cltv_df["monetary_cltv_avg"] = dataframe["Omnichannel_customer_value_total"] / cltv_df["frequency"]

    cltv_df = cltv_df[(cltv_df['frequency'] > 1)]
    cltv_df["recency_cltv_weekly"] = cltv_df["recency_cltv_weekly"] / 7
    cltv_df["T_weekly"] = cltv_df["T_weekly"] / 7
    
    bgf = BetaGeoFitter(penalizer_coef=0.001)

    bgf.fit(cltv_df['frequency'],
            cltv_df['recency_cltv_weekly'],
            cltv_df['T_weekly'])

    #Estimate expected purchases from customers within 3 months
    cltv_df["exp_sales_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4 * 3,
                                                            cltv_df['frequency'],
                                                            cltv_df['recency_cltv_weekly'],
                                                            cltv_df['T_weekly'])

    #Estimate expected purchases from customers within 6 months
    cltv_df["exp_sales_6_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4 * 6,
                                                            cltv_df['frequency'],
                                                            cltv_df['recency_cltv_weekly'],
                                                            cltv_df['T_weekly'])
    ggf = GammaGammaFitter(penalizer_coef=0.01)

    ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])

    cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                                 cltv_df['monetary_cltv_avg'])
    cltv_df.index = cltv_df["customer_id"]
    cltv_df.drop("customer_id", inplace=True, axis=1)
    
    cltv_df["cltv"] = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time=projection_time, # 6 month
                                   freq="W",
                                   discount_rate=0.01)

    print(cltv_df.sort_values(by="cltv", ascending=False).head(20))
    cltv_df["segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])
    print(cltv_df.groupby("segment").agg({"count", "mean", "sum", "min", "max"}))
    return cltv_df